In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade --no-cache-dir gdown

In [3]:
!gdown --id 1Ilfihfk2QxOcgikb2ihNO1YakXUCIPMe

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Ilfihfk2QxOcgikb2ihNO1YakXUCIPMe
From (redirected): https://drive.google.com/uc?id=1Ilfihfk2QxOcgikb2ihNO1YakXUCIPMe&confirm=t&uuid=847ce8be-b61c-4f17-95b1-ab6b9fc212cf
To: /kaggle/working/increased data.zip
100%|██████████████████████████████████████| 1.38G/1.38G [00:25<00:00, 54.7MB/s]


In [4]:
from zipfile import ZipFile

file_name = "/kaggle/working/increased data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [5]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [6]:
#get the classnames  programmatically
import pathlib
data_dir=pathlib.Path("/kaggle/working/data/train")
class_names=np.array(sorted([item.name for item in data_dir.glob("*")])) # created a list of class_names from the sub directories
print(class_names)

[]


In [7]:
#visualize 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import random

def view_random_image(target_dir,target_class):
  target_folder=target_dir+target_class

  random_image=random.sample(os.listdir(target_folder),1)
  img=mpimg.imread(target_folder+"/"+random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image shape : {img.shape}")
  return img

In [8]:
# Define the number of classes in the dataset
num_classes = 10

train_dir='/kaggle/working/data/train'
test_dir='/kaggle/working/data/test'

# Load and preprocess the data
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)
# Load data in from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical') # changed to categorical

test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 10495 images belonging to 10 classes.
Found 3947 images belonging to 10 classes.


In [9]:
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20, # note: this is an int not a float
                                             width_shift_range=0.1,
                                             height_shift_range=0.1,
                                             zoom_range=0.1,
                                             horizontal_flip=True)

train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                  target_size=(224, 224),
                                                                  batch_size=32,
                                                                  class_mode='categorical')

Found 10495 images belonging to 10 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
model_26 = Sequential([

Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
AveragePooling2D(pool_size=(2,2)),

Conv2D(64, (3,3), activation='relu'),
AveragePooling2D(pool_size=(2,2)),

Conv2D(128, (3,3), activation='relu'),
AveragePooling2D(pool_size=(2,2)),
    
Conv2D(256, (3,3), activation='relu'),
AveragePooling2D(pool_size=(2,2)),



# Flatten layer
Flatten(),

# Dense layers
Dense(512, activation='relu'),
Dropout(0.35),

Dense(10, activation='softmax')])

model_26.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                metrics=["accuracy"]) 


In [11]:
model_26.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 average_pooling2d (AverageP  (None, 111, 111, 32)     0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 average_pooling2d_1 (Averag  (None, 54, 54, 64)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 average_pooling2d_2 (Averag  (None, 26, 26, 128)      0

In [12]:
model_26.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model
history = model_26.fit(
    train_data_augmented,
    steps_per_epoch=len(train_data_augmented),
    epochs=200,
    validation_data=test_data,
    validation_steps=len(test_data)
)

Epoch 1/200
328/328 [==============================] - 156s 446ms/step - loss: 2.1708 - accuracy: 0.1840 - val_loss: 2.1387 - val_accuracy: 0.1996
Epoch 2/200
328/328 [==============================] - 149s 453ms/step - loss: 2.0674 - accuracy: 0.2367 - val_loss: 2.0477 - val_accuracy: 0.2539
Epoch 3/200
328/328 [==============================] - 143s 437ms/step - loss: 2.0154 - accuracy: 0.2581 - val_loss: 2.0172 - val_accuracy: 0.2734
Epoch 4/200
328/328 [==============================] - 145s 443ms/step - loss: 1.9656 - accuracy: 0.2806 - val_loss: 1.9568 - val_accuracy: 0.2888
Epoch 5/200
328/328 [==============================] - 146s 445ms/step - loss: 1.9171 - accuracy: 0.3036 - val_loss: 1.9003 - val_accuracy: 0.3185
Epoch 6/200
328/328 [==============================] - 144s 438ms/step - loss: 1.8902 - accuracy: 0.3222 - val_loss: 1.8648 - val_accuracy: 0.3398
Epoch 7/200
328/328 [==============================] - 145s 441ms/step - loss: 1.8564 - accuracy: 0.3313 - val_loss: 1

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model
import IPython
import os

In [20]:
# Specify the path where you want to save the model in the Kaggle output directory
save_path = '/kaggle/working/model_26.h5'

# Save the model to the specified path
model_26.save(save_path)


In [23]:
from IPython.display import FileLink
FileLink(save_path)

/kaggle/working/model_26.h5

In [22]:
model_26.evaluate(test_data)

124/124 [==============================] - 13s 103ms/step - loss: 1.4128 - accuracy: 0.7223


[1.4128338098526, 0.7223207354545593]